In [ ]:
import polars as pl
import matplotlib.pyplot as plt
from datetime import timedelta

df = pl.read_csv("Tf-Schichtbesetzung PoC.csv")

In [ ]:
len(df), (df["State"] == "New").sum(), (df["State"] == "Closed").sum()

In [ ]:
def to_datetime(col: str) -> pl.Expr:
    return pl.col(col).str.strptime(pl.Datetime, r"%d.%m.%Y %H:%M:%S", strict=False)

df_dt = (
    df.with_columns([
        to_datetime("Activated Date"),
        to_datetime("Closed Date"),
    ])
    .drop_nulls(["Activated Date", "Closed Date"])
)

In [ ]:
import datetime as dt


def assign_offsets(intervals: list[tuple[dt.datetime, dt.datetime]]) -> list[int]:
    """
    Assigns a vertical offset level for each (start, end) interval.
    Overlapping intervals receive different levels.
    """
    levels: list[list[tuple[dt.datetime, dt.datetime]]] = []

    for start, end in intervals:
        placed = False
        for level in levels:
            # Check if last interval in level overlaps
            last_start, last_end = level[-1]
            if end >= last_start and start <= last_end:
                # overlap → try next level
                continue
            else:
                # no overlap → put here
                level.append((start, end))
                placed = True
                break

        if not placed:
            # create new level
            levels.append([(start, end)])

    # Map each interval to its level index
    out = []
    for i, (start, end) in enumerate(intervals):
        for lvl_idx, lvl in enumerate(levels):
            if (start, end) in lvl:
                out.append(lvl_idx)
                break
    return out


def plot_ticket_timeline(df: pl.DataFrame) -> None:
    # Parse dates
    df = df.with_columns([
        to_datetime("Activated Date"),
        to_datetime("Closed Date"),
        pl.col("Assigned To").str.replace(r"<.*>", "").str.strip_chars(),
    ])

    # Keep only tickets with valid date ranges
    df = df.drop_nulls(["Activated Date", "Closed Date"])

    processing_time_expr = pl.col("Closed Date") - pl.col("Activated Date")

    # Drop very long tickets, which are architectual or enabling tickets
    df = df.filter(processing_time_expr < timedelta(days=30))

    devs:list[str] = df["Assigned To"].unique().to_list()
    dev_ypos = {dev: i for i, dev in enumerate(devs)}

    fig, ax = plt.subplots(figsize=(14, 0.7 * len(devs)))

    for icol, dev in enumerate(devs):
        ddf = df.filter(pl.col("Assigned To") == dev).sort("Activated Date")

        # extract intervals
        intervals = [
            (row["Activated Date"], row["Closed Date"])
            for row in ddf.iter_rows(named=True)
        ]

        # assign stacking offsets
        offsets = assign_offsets(intervals)

        # draw
        base_y = dev_ypos[dev] * 5  # spacing between developers

        first = True
        for (start, end), off in zip(intervals, offsets):
            ax.hlines(
                y=base_y + off,
                xmin=start,
                xmax=end,
                linewidth=3,
                color=f"C{icol}",
                label=dev if first else None # type:ignore
            )
            first = None

    fig.legend(ncol=3)

    ax.set_yticks([])
    ax.set_xlabel("Date")
    ax.set_title("Ticket Timelines")
    plt.tight_layout()
    plt.show()


# Usage:
plot_ticket_timeline(df)

In [ ]:
from datetime import datetime, time, date
import numpy as np
import polars as pl

SECONDS_PER_DAY = 24 * 3600

def _is_weekday(d: date) -> bool:
    return d.weekday() < 5  # Mon=0 ... Sun=6

def _day_bounds(dt: datetime) -> tuple[datetime, datetime]:
    start = datetime.combine(dt.date(), time.min)
    end = start + timedelta(days=1)
    return start, end

def working_seconds_between(start: datetime, end: datetime) -> float:
    """
    Exact working seconds between two datetimes, counting only Mon-Fri.
    - Counts full middle business days with numpy.busday_count
    - Adds partial time on the first and last business day
    """
    if start >= end:
        return 0.0

    # Full business days strictly between start.date() and end.date()
    start_next_midnight = datetime.combine(start.date() + timedelta(days=1), time.min)
    end_midnight = datetime.combine(end.date(), time.min)
    full_days = int(np.busday_count(start_next_midnight.date(), end_midnight.date()))
    seconds = full_days * SECONDS_PER_DAY

    # Partial time on the start day (if weekday)
    if _is_weekday(start.date()):
        _, start_day_end = _day_bounds(start)
        seconds += (min(end, start_day_end) - start).total_seconds()

    # Partial time on the end day (if different day and weekday)
    if end.date() != start.date() and _is_weekday(end.date()):
        end_day_start, _ = _day_bounds(end)
        seconds += (end - max(start, end_day_start)).total_seconds()

    # Clamp to the raw total to avoid floating rounding oddities
    return max(0.0, min(seconds, (end - start).total_seconds()))

# 2) Compute working seconds and working days
df_work = (
    df_dt.with_columns(
        pl.struct(["Activated Date", "Closed Date"])
          .map_elements(
              lambda s: working_seconds_between(s["Activated Date"], s["Closed Date"]),
              return_dtype=pl.Float64,
          )
          .alias("working_seconds")
    )
    .with_columns((pl.col("working_seconds") / SECONDS_PER_DAY).alias("working_days"))
)

# 3) Median & mean (working days)
median_days = df_work.select(pl.col("working_days").median()).item()
mean_days   = df_work.select(pl.col("working_days").mean()).item()

print(f"median processing time (working days): {median_days:.1f}")
print(f"mean processing time   (working days): {mean_days:.1f}")

# per-developer summary
df_per_dev = (
    df_work
    .with_columns(
        pl.col("Assigned To").str.replace(r"<.*>", "").str.strip_chars().alias("Developer")
    )
    .group_by("Developer")
    .agg(
        pl.col("working_days").median().alias("median_days"),
        pl.col("working_days").mean().alias("mean_days"),
        pl.len().alias("n_tickets"),
    )
    .sort("mean_days")
)
print(df_per_dev)